In [1]:
import numpy as np
import os

In [2]:
def haar_lwt_1d_decompose(data):
    """
    Performs a single level 1D Haar Lifting Wavelet Transform decomposition.

    Args:
        data (np.ndarray): The 1D input data array.

    Returns:
        tuple: A tuple containing:
            - approximation (np.ndarray): The approximation coefficients.
            - detail (np.ndarray): The detail coefficients.
            - original_len (int): The original length of the input data before padding.
    """
    original_len = len(data)
    
    # Pad if length is odd to ensure even split
    if original_len % 2 != 0:
        padded_data = np.pad(data, (0, 1), 'constant', constant_values=0)
    else:
        padded_data = data

    # Split: Separate into even and odd indexed samples
    even = padded_data[::2]
    odd = padded_data[1::2]

    # Predict: Calculate detail coefficients (d_j = odd - even)
    detail = odd - even

    # Update: Calculate approximation coefficients (s_j = even + d_j / 2)
    approximation = even + detail / 2

    return approximation, detail, original_len

def haar_lwt_1d_reconstruct(approximation, detail, original_len):
    """
    Reconstructs a 1D signal from its single level Haar Lifting Wavelet Transform coefficients.

    Args:
        approximation (np.ndarray): The approximation coefficients.
        detail (np.ndarray): The detail coefficients.
        original_len (int): The original length of the signal before decomposition padding.

    Returns:
        np.ndarray: The reconstructed 1D signal.
    """
    # Inverse Update
    even = approximation - detail / 2

    # Inverse Predict
    odd = detail + even

    # Merge: Combine the reconstructed even and odd parts
    combined_padded_len = len(even) + len(odd)
    reconstructed = np.empty(combined_padded_len, dtype=float)
    reconstructed[::2] = even
    reconstructed[1::2] = odd
    
    # Trim to original length
    return reconstructed[:original_len]

def haar_lwt_2d_decompose(data, level):
    """
    Performs an N-level 2D Haar Lifting Wavelet Transform decomposition.

    Args:
        data (np.ndarray): The 2D input data array (e.g., an image).
        level (int): The number of decomposition levels to perform.

    Returns:
        dict: A dictionary containing the final approximation sub-band and
              all detail sub-bands (LH, HL, HH) for each level.
              Keys will be 'LL', 'LH_L1', 'HL_L1', 'HH_L1', 'LH_L2', etc.
              Each value is an np.ndarray.
    """
    if data.ndim != 2:
        raise ValueError("Input data must be 2-dimensional for 2D LWT.")

    current_approx = np.array(data, dtype=float)
    coeffs_tree = {} # Stores all sub-bands

    # Store original shape for reconstruction
    coeffs_tree['original_shape'] = data.shape

    for i in range(1, level + 1):
        # --- Step 1: Apply 1D LWT along rows (axis=1) ---
        # Initialize arrays to store row-wise approx and detail
        rows_approx = np.empty((current_approx.shape[0], (current_approx.shape[1] + 1) // 2), dtype=float)
        rows_detail = np.empty((current_approx.shape[0], (current_approx.shape[1] + 1) // 2), dtype=float)
        
        # Store original row lengths for reconstruction
        original_row_lengths = []

        for r in range(current_approx.shape[0]):
            row_approx, row_detail, original_len = haar_lwt_1d_decompose(current_approx[r, :])
            rows_approx[r, :len(row_approx)] = row_approx # Ensure correct assignment for padded/smaller arrays
            rows_detail[r, :len(row_detail)] = row_detail
            original_row_lengths.append(original_len)
        
        # Store original row lengths for this level
        coeffs_tree[f'original_row_lengths_L{i}'] = original_row_lengths

        # --- Step 2: Apply 1D LWT along columns (axis=0) to rows_approx and rows_detail ---
        # Transpose to apply 1D LWT along columns
        cols_approx_from_rows_approx = np.empty(((rows_approx.shape[0] + 1) // 2, rows_approx.shape[1]), dtype=float)
        cols_detail_from_rows_approx = np.empty(((rows_approx.shape[0] + 1) // 2, rows_approx.shape[1]), dtype=float)
        
        cols_approx_from_rows_detail = np.empty(((rows_detail.shape[0] + 1) // 2, rows_detail.shape[1]), dtype=float)
        cols_detail_from_rows_detail = np.empty(((rows_detail.shape[0] + 1) // 2, rows_detail.shape[1]), dtype=float)

        # Store original column lengths for reconstruction
        original_col_lengths_approx = []
        original_col_lengths_detail = []

        for c in range(rows_approx.shape[1]):
            # For LL and HL (from rows_approx)
            col_approx_ll, col_detail_hl, original_len_col_approx = haar_lwt_1d_decompose(rows_approx[:, c])
            cols_approx_from_rows_approx[:len(col_approx_ll), c] = col_approx_ll
            cols_detail_from_rows_approx[:len(col_detail_hl), c] = col_detail_hl
            original_col_lengths_approx.append(original_len_col_approx)

            # For LH and HH (from rows_detail)
            col_approx_lh, col_detail_hh, original_len_col_detail = haar_lwt_1d_decompose(rows_detail[:, c])
            cols_approx_from_rows_detail[:len(col_approx_lh), c] = col_approx_lh
            cols_detail_from_rows_detail[:len(col_detail_hh), c] = col_detail_hh
            original_col_lengths_detail.append(original_len_col_detail)
        
        # Store original column lengths for this level
        coeffs_tree[f'original_col_lengths_approx_L{i}'] = original_col_lengths_approx
        coeffs_tree[f'original_col_lengths_detail_L{i}'] = original_col_lengths_detail

        # Assign sub-bands for the current level
        # LL is the approximation for the next level
        current_approx = cols_approx_from_rows_approx
        coeffs_tree[f'LH_L{i}'] = cols_approx_from_rows_detail # Low-pass on rows, High-pass on columns
        coeffs_tree[f'HL_L{i}'] = cols_detail_from_rows_approx # High-pass on rows, Low-pass on columns
        coeffs_tree[f'HH_L{i}'] = cols_detail_from_rows_detail # High-pass on rows, High-pass on columns
        
        # Break if the approximation sub-band becomes too small for further decomposition
        if current_approx.shape[0] < 2 or current_approx.shape[1] < 2:
            print(f"Warning: Stopped 2D decomposition at level {i} because approximation sub-band became too small: {current_approx.shape}")
            break
    
    # The final approximation sub-band
    coeffs_tree['LL'] = current_approx
    return coeffs_tree

def haar_lwt_2d_reconstruct(coeffs_tree):
    """
    Reconstructs a 2D signal from its N-level Haar Lifting Wavelet Transform coefficients.

    Args:
        coeffs_tree (dict): A dictionary containing the final approximation sub-band
                            and all detail sub-bands (LH, HL, HH) for each level,
                            as returned by `haar_lwt_2d_decompose`.

    Returns:
        np.ndarray: The reconstructed 2D signal.
    """
    current_reconstruction = coeffs_tree['LL']
    original_full_shape = coeffs_tree['original_shape']
    
    # Determine the number of levels from the keys in coeffs_tree
    levels = 0
    for key in coeffs_tree:
        if key.startswith('LH_L'):
            levels = max(levels, int(key.split('_L')[1]))
    
    # Reconstruct level by level, from coarsest to finest
    for i in range(levels, 0, -1):
        lh = coeffs_tree[f'LH_L{i}']
        hl = coeffs_tree[f'HL_L{i}']
        hh = coeffs_tree[f'HH_L{i}']
        
        original_col_lengths_approx = coeffs_tree[f'original_col_lengths_approx_L{i}']
        original_col_lengths_detail = coeffs_tree[f'original_col_lengths_detail_L{i}']
        original_row_lengths = coeffs_tree[f'original_row_lengths_L{i}']

        # --- Inverse Step 2: Reconstruct columns (axis=0) ---
        # Reconstruct rows_approx from current_reconstruction (LL from previous level) and HL
        reconstructed_rows_approx = np.empty((max(original_col_lengths_approx), current_reconstruction.shape[1]), dtype=float)
        for c in range(current_reconstruction.shape[1]):
            reconstructed_rows_approx[:, c] = haar_lwt_1d_reconstruct(
                current_reconstruction[:, c], hl[:, c], original_col_lengths_approx[c]
            )
        
        # Reconstruct rows_detail from LH and HH
        reconstructed_rows_detail = np.empty((max(original_col_lengths_detail), lh.shape[1]), dtype=float)
        for c in range(lh.shape[1]):
            reconstructed_rows_detail[:, c] = haar_lwt_1d_reconstruct(
                lh[:, c], hh[:, c], original_col_lengths_detail[c]
            )

        # --- Inverse Step 1: Reconstruct rows (axis=1) ---
        # Merge reconstructed_rows_approx and reconstructed_rows_detail
        # This is the tricky part: we need to merge the approx and detail *matrices*
        # The 1D reconstruct function expects 1D arrays.
        # We need to apply the 1D reconstruct function row-wise to combine
        # `reconstructed_rows_approx` (which is the 'even' part conceptually)
        # and `reconstructed_rows_detail` (which is the 'detail' part conceptually).
        
        # The shape of the output of this step should be the original shape of the
        # data at this level before the row-wise decomposition.
        
        # The number of rows in reconstructed_rows_approx and reconstructed_rows_detail should be the same
        # This will be the height of the image at this level.
        reconstructed_current_level = np.empty((reconstructed_rows_approx.shape[0], max(original_row_lengths)), dtype=float)

        for r in range(reconstructed_rows_approx.shape[0]):
            # The 'approximation' input to 1D reconstruct is the 'even' part (reconstructed_rows_approx[r, :])
            # The 'detail' input to 1D reconstruct is the 'odd - even' part (reconstructed_rows_detail[r, :])
            # This is where the inverse lifting steps (even = s - d/2, odd = d + even) are applied.
            
            # The `haar_lwt_1d_reconstruct` already handles the merging logic.
            # We need to treat `reconstructed_rows_approx` as the approximation part
            # and `reconstructed_rows_detail` as the detail part for the row-wise reconstruction.
            
            # This is a conceptual mapping. The `haar_lwt_1d_reconstruct` takes (approx, detail, original_len).
            # Here, `reconstructed_rows_approx` acts as the 's_j' and `reconstructed_rows_detail` acts as 'd_j'.
            
            # The `haar_lwt_1d_reconstruct` expects the *approximation* and *detail* coefficients
            # from a *single level* 1D decomposition.
            # So, we need to apply the inverse of the row-wise transform.
            # This means taking the `reconstructed_rows_approx` (which is effectively LL/HL)
            # and `reconstructed_rows_detail` (which is effectively LH/HH)
            # and combining them row by row.

            # The `haar_lwt_1d_reconstruct` function needs the approximation and detail coefficients
            # that were *directly* produced by a 1D decomposition.
            # The `reconstructed_rows_approx` and `reconstructed_rows_detail` are already the result
            # of column-wise reconstruction. They are not the direct 'approximation' and 'detail'
            # for the row-wise inverse.

            # Let's rethink the 2D reconstruction.
            # After column-wise reconstruction, we have `reconstructed_rows_approx` (which is the A part of the row-wise transform)
            # and `reconstructed_rows_detail` (which is the D part of the row-wise transform).
            # We need to merge these two 2D arrays back into the original 2D array for this level.

            # The `haar_lwt_1d_reconstruct` takes `approximation`, `detail`, `original_len`.
            # We need to apply this row by row.
            # `reconstructed_rows_approx[r, :]` is the approximation part for row `r`.
            # `reconstructed_rows_detail[r, :]` is the detail part for row `r`.
            reconstructed_current_level[r, :] = haar_lwt_1d_reconstruct(
                reconstructed_rows_approx[r, :],
                reconstructed_rows_detail[r, :],
                original_row_lengths[r]
            )
        
        current_reconstruction = reconstructed_current_level

    # The final reconstruction needs to be trimmed to the original full shape
    return current_reconstruction[:original_full_shape[0], :original_full_shape[1]]


def save_coefficients_to_files(output_dir, coeffs_tree):
    """
    Saves all Haar LWT coefficients (approximation and all detail levels) into a single .csv file,
    with each coefficient type in its own column.

    Args:
        output_dir (str): The directory where the coefficient file will be saved.
                          If the directory does not exist, it will be created.
        coeffs_tree (dict): A dictionary containing the final approximation sub-band
                            and all detail sub-bands (LH, HL, HH) for each level,
                            as returned by `haar_lwt_2d_decompose`.
    """
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Prepare a list of all flattened coefficient arrays and their headers
    all_coeff_arrays_flat = []
    headers = []

    # Add the final approximation (LL)
    if 'LL' in coeffs_tree:
        all_coeff_arrays_flat.append(coeffs_tree['LL'].flatten())
        headers.append('LL_Approximation')

    # Add detail coefficients for each level
    levels = 0
    for key in coeffs_tree:
        if key.startswith('LH_L'):
            levels = max(levels, int(key.split('_L')[1]))

    for i in range(1, levels + 1):
        if f'LH_L{i}' in coeffs_tree:
            all_coeff_arrays_flat.append(coeffs_tree[f'LH_L{i}'].flatten())
            headers.append(f'LH_L{i}')
        if f'HL_L{i}' in coeffs_tree:
            all_coeff_arrays_flat.append(coeffs_tree[f'HL_L{i}'].flatten())
            headers.append(f'HL_L{i}')
        if f'HH_L{i}' in coeffs_tree:
            all_coeff_arrays_flat.append(coeffs_tree[f'HH_L{i}'].flatten())
            headers.append(f'HH_L{i}')

    # Find the maximum length among all flattened coefficient arrays for consistent column size
    max_len = 0
    if all_coeff_arrays_flat:
        max_len = max(len(arr) for arr in all_coeff_arrays_flat)

    # Pad shorter arrays with NaN to match the maximum length
    padded_coeff_arrays = []
    for arr in all_coeff_arrays_flat:
        if len(arr) < max_len:
            padded_arr = np.pad(arr, (0, max_len - len(arr)), 'constant', constant_values=np.nan)
        else:
            padded_arr = arr
        padded_coeff_arrays.append(padded_arr)

    # Stack the padded arrays horizontally to form a 2D array (columns)
    if padded_coeff_arrays:
        combined_coeffs_2d = np.column_stack(padded_coeff_arrays)
    else:
        combined_coeffs_2d = np.array([[]]) # Handle case where no coefficients are generated

    # Create the header string for the CSV file
    header_str = ','.join(headers)

    # Save the combined coefficients to a single CSV file
    combined_filepath = os.path.join(output_dir, 'all_haar_lwt_coeffs_columns_nd.csv')
    np.savetxt(combined_filepath, combined_coeffs_2d, delimiter=',', header=header_str, comments='')
    print(f"Saved all Haar LWT coefficients (in columns) to: {combined_filepath}")

# --- Example Usage ---
if __name__ == "__main__":
    # 1. Define sample 2D data (e.g., a small image)
    # Example with even dimensions
    data_2d_even = np.array([
        [10, 20, 30, 40],
        [50, 60, 70, 80],
        [90, 100, 110, 120],
        [130, 140, 150, 160]
    ])
    # Example with odd dimensions
    data_2d_odd = np.array([
        [1, 2, 3, 4, 5],
        [6, 7, 8, 9, 10],
        [11, 12, 13, 14, 15],
        [16, 17, 18, 19, 20],
        [21, 22, 23, 24, 25]
    ])
    
    # Choose which data to use for demonstration
    input_data_2d = data_2d_odd
    n_levels_2d = 2 # Number of decomposition levels

    print(f"Original 2D Data:\n{input_data_2d}")
    print(f"Original 2D Data Shape: {input_data_2d.shape}")
    print(f"Decomposition Levels: {n_levels_2d}")
    print("-" * 30)

    # 2. Perform N-level 2D Haar LWT decomposition
    coeffs_2d = haar_lwt_2d_decompose(input_data_2d, n_levels_2d)

    print("\n--- 2D Decomposition Results ---")
    print(f"Final LL (Approximation) Sub-band:\n{coeffs_2d['LL']}")
    for i in range(1, n_levels_2d + 1):
        if f'LH_L{i}' in coeffs_2d:
            print(f"LH_L{i} (Horizontal Detail Level {i}):\n{coeffs_2d[f'LH_L{i}']}")
        if f'HL_L{i}' in coeffs_2d:
            print(f"HL_L{i} (Vertical Detail Level {i}):\n{coeffs_2d[f'HL_L{i}']}")
        if f'HH_L{i}' in coeffs_2d:
            print(f"HH_L{i} (Diagonal Detail Level {i}):\n{coeffs_2d[f'HH_L{i}']}")
    print("-" * 30)

    # 3. Save coefficients to files
    output_directory_2d = 'haar_lwt_coeffs_2d'
    save_coefficients_to_files(output_directory_2d, coeffs_2d)
    print("-" * 30)

    # 4. Perform 2D reconstruction
    reconstructed_data_2d = haar_lwt_2d_reconstruct(coeffs_2d)

    print("\n--- 2D Reconstruction Results ---")
    print(f"Reconstructed Data:\n{reconstructed_data_2d}")
    
    # 5. Verify 2D reconstruction accuracy
    is_reconstruction_accurate_2d = np.allclose(input_data_2d, reconstructed_data_2d)
    print(f"Is 2D Reconstruction Accurate? {is_reconstruction_accurate_2d}")
    if not is_reconstruction_accurate_2d:
        print(f"Difference:\n{input_data_2d - reconstructed_data_2d}")
    print("-" * 30)

    print("\n--- Generalizing to N-Dimensional Data (Conceptual) ---")
    print("For N-dimensional data beyond 2D (e.g., 3D volumes), the principle remains the same:")
    print("You would recursively apply the 1D LWT along each dimension.")
    print("For example, for 3D data, you would perform 1D LWT along axis 0, then axis 1, then axis 2.")
    print("This would generate $2^N$ sub-bands at each level (e.g., 8 sub-bands for 3D: LLL, LLH, LHL, LHH, HLL, HLH, HHL, HHH).")
    print("The complexity of managing these sub-bands and their reconstruction increases significantly.")
    print("The current 2D implementation provides a solid foundation for understanding this generalization.")
    print("-" * 30)


Original 2D Data:
[[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]
 [16 17 18 19 20]
 [21 22 23 24 25]]
Original 2D Data Shape: (5, 5)
Decomposition Levels: 2
------------------------------

--- 2D Decomposition Results ---
Final LL (Approximation) Sub-band:
[[10.      3.125 ]
 [ 5.625   1.5625]]
LH_L1 (Horizontal Detail Level 1):
[[  1.    1.   -7.5]
 [  1.    1.  -17.5]
 [  0.5   0.5 -12.5]]
HL_L1 (Vertical Detail Level 1):
[[  5.    5.    2.5]
 [  5.    5.    2.5]
 [-21.5 -23.5 -12.5]]
HH_L1 (Diagonal Detail Level 1):
[[ 0.  0. -5.]
 [ 0.  0. -5.]
 [-1. -1. 25.]]
LH_L2 (Horizontal Detail Level 2):
[[ 2.    -6.25 ]
 [ 0.5   -3.125]]
HL_L2 (Vertical Detail Level 2):
[[ 10.      2.5  ]
 [-11.25   -3.125]]
HH_L2 (Diagonal Detail Level 2):
[[ 0.   -5.  ]
 [-1.    6.25]]
------------------------------
Saved all Haar LWT coefficients (in columns) to: haar_lwt_coeffs_2d/all_haar_lwt_coeffs_columns_nd.csv
------------------------------

--- 2D Reconstruction Results ---
Reconstructed Da